# Bayesian Radial Velocity Planet Search

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn; seaborn.set() #nice formatting

Two sources of equations:

- Balan 2009: http://adsabs.harvard.edu/abs/2009MNRAS.394.1936B
- Exofit: http://www.star.ucl.ac.uk/~lahav/ExoFitv2.pdf

- http://arxiv.org/pdf/1401.6128.pdf

Equation for radial velocity:

$$
v(t_i) = K[ \sin(f_i + \omega) + e \sin(\omega)]
$$

$$
K = \frac{m_p}{m_s + m_p} \frac{2\pi}{T}\frac{a \sin i}{\sqrt{1 - e^2}}
$$

The true anomaly $f$ satisfies

$$
\cos(f_i) = \frac{\cos(E_i) - e}{1 - e\cos E_i}
$$

Rearranging this we can write
$$
f_i = 2 \cdot{\rm atan2}\left(\sqrt{1 + e}\sin(E_i/2), \sqrt{1 - e} \cos(E_i/2)\right)
$$

The eccentric anomaly $E$ satisfies
$$
M_i = E_i - e\sin E_i
$$

and the mean anomaly is
$$
M_i = \frac{2\pi}{T}(t_i + \tau)
$$

and $\tau$ is the time of pericenter passage, which we'll parametrize with the parameter $\chi = \tau /  T$

Parameters needed to compute the radial velocity:

- $T$: orbital period
- $K$: amplitude of oscillation
- $V$: offset of oscillation
- $e$: eccentricity
- $\omega$: longitude of periastron
- $\chi$: dimensionless phase offset

Additionally, we will fit a scatter parameter $s$ which accounts for data uncertainties not reflected in the reported errors.

In [ ]:
from scipy import optimize

@np.vectorize
def compute_E(M, e):
    """Solve Kepler's eqns for eccentric anomaly"""
    f = lambda E, M=M, e=e: E - e * np.sin(E) - M
    return optimize.brentq(f, 0, 2 * np.pi)


def radial_velocity(t, theta):
    """Compute radial velocity given orbital parameters"""
    T, e, K, V, omega, chi = theta[:6]
    
    # compute mean anomaly
    M = 2 * np.pi * ((t / T + chi) % 1)
    
    # solve for eccentric anomaly
    E = compute_E(M, e)
    
    # compute true anomaly
    f = 2 * np.arctan2(np.sqrt(1 + e) * np.sin(E / 2),
                       np.sqrt(1 - e) * np.cos(E / 2))
    
    # compute velocity
    return V - K * (np.sin(f + omega) + e * np.sin(omega))

Visualize this with IPython interact, to make sure the results make sense:

In [ ]:
from ipywidgets import interact

def plot_RV(T, e, K, V, omega, chi):
    t = np.linspace(0, 5, 200)
    theta = [T, e, K, V, omega, chi]
    plt.plot(t, radial_velocity(t, theta))
    
interact(plot_RV,
         T=(0, 5.), K=(0, 2000.), V=(-2000., 2000.),
         e=(0, 1.), omega=(0, 2 * np.pi), chi=(0, 1.));

In [ ]:
theta_names = ['T', 'e', 'K', 'V', 'omega', 'chi', 's']
theta_sim = [700, 0.38, 60, 12, 3.10, 0.67, 1]
Nobs = 50

rng = np.random.RandomState(0)
t_sim = 1400 + 600 * rng.rand(Nobs)
err_sim = 1 + rng.rand(Nobs)
rv_sim = radial_velocity(t_sim, theta_sim) + err_sim * rng.randn(Nobs)

plt.errorbar(t_sim, rv_sim, err_sim, fmt='.k');
xlim = plt.xlim()
t_fit = np.linspace(xlim[0], xlim[1], 500)
plt.plot(t_fit, radial_velocity(t_fit, theta_sim), color='gray');

## Bayesian Fit

In [ ]:
theta_min = np.array([0.2, 0, 0.01, -2000, 0, 0, 0.01])
theta_max = np.array([2000, 1, 2000, 2000, 2 * np.pi, 1, 100])

def log_prior(theta):
    if np.any(theta < theta_min) or np.any(theta > theta_max):
        return -np.inf # log(0)
    
    # Jeffreys Prior on T, K, and s
    return -np.sum(np.log(theta[[0, 1, 6]]))

def log_likelihood(theta, t, rv, rv_err):
    s = theta[6]
    sq_err = rv_err ** 2 + s ** 2
    rv_model = radial_velocity(t, theta)
    return -0.5 * np.sum(np.log(sq_err) + (rv - rv_model) ** 2 / sq_err)

def log_posterior(theta, t, rv, rv_err):
    ln_prior = log_prior(theta)
    if np.isinf(ln_prior):
        return ln_prior
    else:
        return ln_prior + log_likelihood(theta, t, rv, rv_err)        

In [ ]:
import emcee

ndim = len(theta_sim)  # number of parameters in the model
nwalkers = 50  # number of MCMC walkers

# start with a tight distribution of theta around the initial guess
rng = np.random.RandomState(42)
theta_guess = 0.5 * (theta_min + theta_max)
theta_range = (theta_max - theta_min)
starting_guesses = theta_guess + 0.05 * theta_range * rng.randn(nwalkers, ndim)

# create the sampler; fix the random state for replicability
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(t_sim, rv_sim, err_sim))
sampler.random_state = rng

# time and run the MCMC
%time pos, prob, state = sampler.run_mcmc(starting_guesses, 1000)

In [ ]:
def plot_chains(sampler):
    fig, ax = plt.subplots(7, figsize=(8, 10), sharex=True)
    for i in range(7):
        ax[i].plot(sampler.chain[:, :, i].T, '-k', alpha=0.2);
        ax[i].set_ylabel(theta_names[i])

plot_chains(sampler)

Yikes! it's all over the place! The issue here is that our initialization was haphazard; we can do much better if we more carefully initialize our function.

First, let's use a Lomb-Scargle periodogram to find a suitable guess at the period.
The [gatspy](http://www.astroml.org/gatspy/) package has a nice implementation:

In [ ]:
from gatspy.periodic import LombScargleFast
model = LombScargleFast()
model.optimizer.set(period_range=(1, 2000), quiet=True)
model.fit(t_sim, rv_sim, err_sim)
model.best_period

In [ ]:
# starting guess: some data-driven, some just in middle of prior range
rv_range = 0.5 * (np.max(rv_sim) - np.min(rv_sim))
rv_center = np.mean(rv_sim)
theta_guess = [model.best_period, 0.5, rv_range, rv_center, 3.14, 0.5, err_sim.mean()]

In [ ]:
sampler.reset()
start = theta_guess * (1 + 0.01 * rng.randn(nwalkers, ndim))
pos, prob, state = sampler.run_mcmc(start, 1000)

In [ ]:
plot_chains(sampler)

Much more reasonable! The trace appears to have stabilized by the end of this, so let's reset and get a clean 1000 samples from the stabilie:

In [ ]:
sampler.reset()
pos, prob, state = sampler.run_mcmc(pos, 1000)

In [ ]:
import corner
corner.corner(sampler.flatchain, labels=theta_names, truths=theta_sim);

We can look at the results by sampling from the posterior:

In [ ]:
plt.errorbar(t_sim, rv_sim, err_sim, fmt='.k')
ylim = plt.ylim()
rng = np.random.RandomState(42)

for i in rng.choice(sampler.flatchain.shape[0], 200):
    rv_fit = radial_velocity(t_fit, sampler.flatchain[i])
    plt.plot(t_fit, rv_fit, '-', alpha=0.01)
plt.ylim(ylim);

Let's pull-out the period and eccentricity, marginalizing over all other parameters:

In [ ]:
import corner
corner.corner(sampler.flatchain[:, :2], 
              labels=theta_names[:2],
              truths=theta_sim[:2]);

We have found a marginalized posterior distribution for the period and eccentricity which contains our our input parameters.